## SunSPoT Steps: 
This file explains the process of generating load profile and estimating the bill saving based on the user inputs.

Prepared by Navid Haghdadi March 2019


In [1]:
# Import modules for running this notebook
import numpy as np
import pandas as pd
import pickle
import json
import os
import requests
import plotly # only necessary for plotting
from plotly import tools # only necessary for plotting
import plotly.graph_objs as go # only necessary for plotting
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot # only necessary for plotting
init_notebook_mode(connected=True) # only necessary for plotting

When a user starts the process (by intreacting with SunSPoT tool), two things should be done by Icelab:

1- A PV profile (hourly) for a typical year (using Jessie's python script), the data should have two columns(timestamp, and PV performance (kW/kWp) (we assume here it is saved in `PVProfile_Example.csv` 

2- A user input file with inputs from the user. An example is saved as `user_inputs_Example.json` and should be in the following format:

`user_inputs = {'postcode': 2033, 'lat': -30, 'long': 150, 'load_profile_provided': 'yes', 'family_size': 2, 'poolpump': 'yes', 'controlled_load': 'yes', 'AC_type': 'Split', 'dwell': 'SeparateHouse', 'smart_meter': 'yes', 'dryer_usage': 'medium',  'HAS_GAS_HEATING': 'yes',  'HAS_GAS_HOT_WATER': 'yes', 'HAS_GAS_COOKING': 'yes','NUM_ROOMS_HEATED': 1, 'NUM_REFRIGERATORS': 1, 'previous_usage': {'Bill 1': {'total': 'N/A', 'peak': 100, 'offpeak': 100, 'shoulder': 100, 'start_date': '2018-02-02', 'end_date': '2018-05-01'}, 'Bill 2': {'total': 'N/A', 'peak': 120, 'offpeak': 110, 'shoulder': 50, 'start_date': '2018-01-01', 'end_date': '2018-02-01'}}}`

User input options:

`postcode`: a valid postcode in Australia (perhaps match against a list..)

`lat`: Icelab to generate based on the user's location

`long`: Icelab to generate based on the user's location

`load_profile_provided`: yes / no

`family_size`: 1, 2, 3, 4+

`poolpump`: yes / no

`dryer_usage`: high/medium/low/no

`controlled_load`: yes / no

`AC_type`: Split, Ducted, NoAirCon, OtherAirCon

`dwell`: SeparateHouse, SemiDetached, Unit

`smart_meter`: yes / no

`previous_usage` : if clicked expand the first entry and if click + expand the next one and so on.

an example is provided in `user_inputs_Example.json`

# Running the function

The main function is `saving_est` inside the `saving_estimator.py`

the inputs are:

`user_inputs` : Provided by Icalab based on user's input

`pv_profile` : provided by another python script for PV generation estimates

`selected_tariff` : selected by user from a list of tariffs provided by icelab

`pv_size_kw` : provided by icelab based on the area the user selected

`Battery_Size` : kW and kWh (assume C1 battery). We should find a way to "suggest" size or perhaps brands to the user. 

In [2]:
# Importing the saving estimator for testing:
from saving_estimator import saving_est

In [3]:
# 1- user inputs:
user_inputs = json.load(open('user_inputs_Default.json'))
# The default values are saved in user_inputs_Default.json. 
# The "user_inputs" should be the same as this file unless the user change anything
# Below is the list of them so we can change them like the user

#  changing user input for testing purpose:
# user_inputs['load_profile_provided']='no'
# user_inputs['family_size']= 2
# user_inputs['poolpump']='no'
# user_inputs['controlled_load']= 'no'
# user_inputs['AC_type']= 'Ducted'
# user_inputs['dwell']= 'SeparateHouse'
# user_inputs['smart_meter']= 'yes'
# user_inputs['dryer_usage']= 'medium'
# user_inputs['HAS_GAS_HEATING']= 'no'
# user_inputs['HAS_GAS_HOT_WATER']= 'no'
# user_inputs['HAS_GAS_COOKING']= 'no'
# user_inputs['NUM_ROOMS_HEATED']= 3
# user_inputs['NUM_REFRIGERATORS']= 1
# user_inputs['previous_usage']['Bill 1']['total']= 'N/A'
# user_inputs['previous_usage']['Bill 1']['peak']= 800
# user_inputs['previous_usage']['Bill 1']['offpeak']= 500
# user_inputs['previous_usage']['Bill 1']['shoulder']= 300
# user_inputs['previous_usage']['Bill 1']['start_date']= '2018-02-02'
# user_inputs['previous_usage']['Bill 1']['end_date']= '2018-05-01'
# user_inputs['previous_usage']['Bill 2']['total']= 'N/A'
# user_inputs['previous_usage']['Bill 2']['peak']= 300
# user_inputs['previous_usage']['Bill 2']['offpeak']= 200
# user_inputs['previous_usage']['Bill 2']['shoulder']= 100
# user_inputs['previous_usage']['Bill 2']['start_date']= '2018-01-01'
# user_inputs['previous_usage']['Bill 2']['end_date']= '2018-02-01'
# user_inputs['previous_usage']=''      # or no previous bill provided
display(user_inputs)

# if user provided load profile (here we assume it is saved in csv file)
user_input_load_profile= pd.read_csv('SelfProvided_LoadProfile_Example.csv')

{'postcode': 2033,
 'lat': -30,
 'long': 150,
 'load_profile_provided': 'yes',
 'family_size': 2,
 'poolpump': 'no',
 'controlled_load': 'no',
 'AC_type': 'Split',
 'dwell': 'SeparateHouse',
 'smart_meter': 'yes',
 'dryer_usage': 'medium',
 'HAS_GAS_HEATING': 'yes',
 'HAS_GAS_HOT_WATER': 'yes',
 'HAS_GAS_COOKING': 'yes',
 'NUM_ROOMS_HEATED': 1,
 'NUM_REFRIGERATORS': 1,
 'previous_usage': ''}

In [4]:
#  Preparing the inputs:
# 2- PV profile:
cwd = os.getcwd()
pv_profile = pd.read_csv(os.path.join(cwd, "PVProfile_Example.csv"))
# pv_profile = pd.read_csv(os.path.join(cwd, "PVProfile_Example_3kW.csv"))
pv_profile['TS'] = pd.to_datetime(pv_profile['TS'],format='%d/%m/%Y %H:%M')
# pv_profile.head()

# 3- PV size (only for calculating per kw generation). Icelab to provide based on selected roof area
pv_size_kw=3  # 3 kW

In [5]:
# 4- Tariff:
# Icelab to show a list of tariffs filtered by the location of user based on the distribution_boundaries.geojson file

# Suppose Icelab showed the list and user selected: Origin Flat Rate NSW (Endeavour area)

Tariff_name = "Origin Flat Rate NSW (Endeavour area)"
Tariff_name = "Power Direct TOU NSW"

all_tariffs = requests.get('http://api.ceem.org.au/elec-tariffs/retail')
all_tariffs = all_tariffs.json()
for i in range(len(all_tariffs)):
    if all_tariffs[i]['Name'] == Tariff_name:
        selected_tariff = all_tariffs[i]

display(selected_tariff)

{'Date_accessed': '2018-11-28',
 'Discount (%)': 27,
 'Distributor': 'Endeavour',
 'Name': 'Power Direct TOU NSW',
 'Parameters': {'Daily': {'Unit': '$/day', 'Value': 0.979},
  'Energy': {'Off Peak-weekdays': {'Month': [1,
     2,
     3,
     4,
     5,
     6,
     7,
     8,
     9,
     10,
     11,
     12],
    'TimeIntervals': {'T1': ['22:00', '07:00']},
    'Unit': '$/kWh',
    'Value': 0.1757,
    'Weekday': True,
    'Weekend': False},
   'Off Peak-weekends': {'Month': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    'TimeIntervals': {'T1': ['00:00', '00:00']},
    'Unit': '$/kWh',
    'Value': 0.1757,
    'Weekday': False,
    'Weekend': True},
   'Peak-weekdays': {'Month': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    'TimeIntervals': {'T1': ['13:00', '20:00']},
    'Unit': '$/kWh',
    'Value': 0.4171,
    'Weekday': True,
    'Weekend': False},
   'Shoulder-weekdays': {'Month': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    'TimeIntervals': {'T1': ['07:00', '13:00'], 'T2': ['20:

In [6]:
# 5- Battery info (zero if no battery). Icelab to obtain from user (or we will have a dropdown list of some brands)
battery_kW=5
battery_kWh=5

# Distributor (Icelab to provide from the location of customer based on the distribution_boundaries.geojson file)
distributor='Ausgrid'


In [7]:
#  Running the function to get the result (with battery it takes 13 sec in my PC. without battery it's few seconds)
Results=saving_est(user_inputs, pv_profile, selected_tariff,pv_size_kw,battery_kW,battery_kWh,distributor,user_input_load_profile)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


In [8]:
# #  Showing the results 

# display('Estimated total annual AC output: '+ Results['Annual_PV_Generation'].astype(int).astype(str) + ' kWh')
# display('Estimated annual AC output per kW capacity: '+ Results['Annual_PV_Generation_per_kW'].astype(int).astype(str) + ' kWh')
# display('Estimated net output to grid: '+ Results['Est_Annual_PV_export_SolarOnly'].astype(int).astype(str) + ' kWh')
# display('Estimated saving due to not using the grid: '+ Results['Saving_due_to_not_using_grid_SolarOnly'].astype(int).astype(str) + ' $')
# display('Estimated annual Feed-in tariff rebate: '+ Results['FiT_Payment_SolarOnly'].astype(int).astype(str) + ' $')
# display('Estimated total annual saving: '+ Results['Annual_Saving_SolarOnly'].astype(int).astype(str) + ' $')
# display('Estimated annual avoided CO2: ' + (0.82* Results['Annual_PV_Generation']).astype(int).astype(str)  + ' kg CO2')

# # Estimated daily seasonal load vs PV generation

# LoadSeasonalPattern=pd.read_json(Results['Load_seasonal_pattern_kW'])
# PVSeasonalPattern=pd.read_json(Results['PV_seasonal_pattern_kW'])

# LoadSeasonalPattern_Sum=LoadSeasonalPattern.loc[LoadSeasonalPattern[0]==1,[1,2]]
# LoadSeasonalPattern_Fal=LoadSeasonalPattern.loc[LoadSeasonalPattern[0]==2,[1,2]]
# LoadSeasonalPattern_Win=LoadSeasonalPattern.loc[LoadSeasonalPattern[0]==3,[1,2]]
# LoadSeasonalPattern_Spr=LoadSeasonalPattern.loc[LoadSeasonalPattern[0]==4,[1,2]]
# PVSeasonalPattern_Sum=PVSeasonalPattern.loc[PVSeasonalPattern[0]==1,[1,2]]
# PVSeasonalPattern_Fal=PVSeasonalPattern.loc[PVSeasonalPattern[0]==2,[1,2]]
# PVSeasonalPattern_Win=PVSeasonalPattern.loc[PVSeasonalPattern[0]==3,[1,2]]
# PVSeasonalPattern_Spr=PVSeasonalPattern.loc[PVSeasonalPattern[0]==4,[1,2]]

# trace1 = go.Scatter(
#     x=LoadSeasonalPattern_Sum[1],
#     y=LoadSeasonalPattern_Sum[2],fill='tozeroy', mode= 'none',name='Load (kW)')
# trace11 = go.Scatter(
#     x=PVSeasonalPattern_Sum[1],
#     y=PVSeasonalPattern_Sum[2],fill='tonexty', mode= 'none',name='PV (kW)')
# trace2 = go.Scatter(
#     x=LoadSeasonalPattern_Fal[1],
#     y=LoadSeasonalPattern_Fal[2],
#     xaxis='x2',yaxis='y2',fill='tozeroy', mode= 'none',name='Load (kW)')
# trace21 = go.Scatter(
#     x=PVSeasonalPattern_Fal[1],
#     y=PVSeasonalPattern_Fal[2],
#     xaxis='x2',yaxis='y2',fill='tonexty', mode= 'none',name='PV (kW)')
# trace3 = go.Scatter(
#     x=LoadSeasonalPattern_Win[1],
#     y=LoadSeasonalPattern_Win[2],
#     xaxis='x3',yaxis='y3',fill='tozeroy', mode= 'none',name='Load (kW)')
# trace31 = go.Scatter(
#     x=PVSeasonalPattern_Win[1],
#     y=PVSeasonalPattern_Win[2],
#     xaxis='x3',yaxis='y3',fill='tonexty', mode= 'none',name='PV (kW)')
# trace4 = go.Scatter(
#     x=LoadSeasonalPattern_Spr[1],
#     y=LoadSeasonalPattern_Spr[2],
#     xaxis='x4',yaxis='y4',fill='tozeroy', mode= 'none',name='Load (kW)')
# trace41 = go.Scatter(
#     x=PVSeasonalPattern_Spr[1],
#     y=PVSeasonalPattern_Spr[2],
#     xaxis='x4',yaxis='y4',fill='tonexty', mode= 'none',name='PV (kW)')

# fig = tools.make_subplots(rows=2, cols=2, print_grid=False, subplot_titles=('Summer', 'Autumn',
#                                                           'Winter', 'Spring'))
# fig.append_trace(trace1, 1, 1)
# fig.append_trace(trace11, 1, 1)
# fig.append_trace(trace2, 1, 2)
# fig.append_trace(trace21, 1, 2)
# fig.append_trace(trace3, 2, 1)
# fig.append_trace(trace31, 2, 1)
# fig.append_trace(trace4, 2, 2)
# fig.append_trace(trace41, 2, 2)
# MaxRange=max(max(LoadSeasonalPattern[2]),max(PVSeasonalPattern[2]))
# fig['layout']['yaxis1'].update(title='kW',range=[0,MaxRange])
# fig['layout']['yaxis2'].update(title='kW',range=[0,MaxRange])
# fig['layout']['yaxis3'].update(title='kW',range=[0,MaxRange])
# fig['layout']['yaxis4'].update(title='kW',range=[0,MaxRange])

# fig['layout'].update(height=600, width=1000, title='Estimated daily seasonal load vs PV generation',showlegend=False)
# # fig = go.Figure(data, layout)
# iplot(fig)

# fig = { "data": [ { "values": [Results['Annual_PV_Generation']-Results['Est_Annual_PV_export_SolarOnly'],Results['Est_Annual_PV_export_SolarOnly']],
#       "labels": ["Self consumption", "Export to the grid"],"domain": {"x": [0, .48]},  "text":["Use", "Export"], "name": "PV Generation (kWh)", 
#                    "hoverinfo":"label+percent+value+name",  "hole": .4,  "type": "pie" },  
#                  { "values": [Results['Old_Bill'], Results['Saving_due_to_not_using_grid_SolarOnly'], Results['FiT_Payment_SolarOnly']],
#       "labels": ["New Bill","Saving due to not using the grid", "Feed in tariff rebate"],
#                   "text":["New Bill","Not buying from grid","FiT"],"textposition":"inside","domain": {"x": [.52, 1]},
#                   "name": "Annual Bill","hoverinfo":"label+percent+value+name", "hole": .4,"type": "pie"}],
#        "layout": {"title":"Summary of the result",
#                   "annotations": [{ "showarrow": False, "text": 'PV (kWh)',"x": 0.20,"y": 0.5 },
#             {  "showarrow": False, "text": "Bill ($)", "x": 0.8, "y": 0.5}] }}
# iplot(fig, filename='donut')




In [9]:
# plotting load profile (for internal checks)
LoadProf=pd.read_json(Results['Load_Prof'])
LoadProf[0]=pd.to_datetime(LoadProf[0],unit='ms')
data =[go.Scatter(x= LoadProf[0], y= LoadProf[1],name='Load profile (kWh)', mode='lines')]

layout = go.Layout(title='Load profile' ,xaxis=dict(title='Time'),yaxis=dict(title='Power (kWh)'))
fig = go.Figure(data, layout)
iplot(fig)

LoadProf['Month']=LoadProf[0].dt.month
LoadProf_monthly=LoadProf.groupby('Month').sum()
data =[go.Bar(x= LoadProf_monthly.index, y= LoadProf_monthly[1])]
layout = go.Layout(title='Monthly kWh' ,xaxis=dict(title='Month'),yaxis=dict(title='Energy (kWh)'))
fig = go.Figure(data, layout)
iplot(fig)
display('Annual kWh: ' + LoadProf_monthly[1].sum().astype(int).astype(str) +' kWh')

'Annual kWh: 6462 kWh'

In [10]:
#  Showing the results for the battery

#  if the battery exists
display('Estimated total annual AC output: '+ Results['Annual_PV_Generation'].astype(int).astype(str) + ' kWh')
display('Estimated annual AC output per kW capacity: '+ Results['Annual_PV_Generation_per_kW'].astype(int).astype(str) + ' kWh')
display('Estimated annual avoided CO2: ' + (0.82* Results['Annual_PV_Generation']).astype(int).astype(str)  + ' kg CO2')
display('PV:')

display('Estimated net output to grid (No battery): '+ Results['Est_Annual_PV_export_SolarOnly'].astype(int).astype(str) + ' kWh')
display('Estimated saving due to not using the grid (No battery): '+ Results['Saving_due_to_not_using_grid_SolarOnly'].astype(int).astype(str) + ' $')
display('Estimated annual Feed-in tariff rebate (No battery): '+ Results['FiT_Payment_SolarOnly'].astype(int).astype(str) + ' $')
display('Estimated total annual saving (No battery): '+ Results['Annual_Saving_SolarOnly'].astype(int).astype(str) + ' $')

if battery_kW > 0:
    display('Battery:')
    display('Estimated net output to grid: '+ Results['Est_Annual_PV_export_SolBatt'].astype(int).astype(str) + ' kWh')
    display('Estimated saving due to not using the grid: '+ Results['Saving_due_to_not_using_grid_SolBatt'].astype(int).astype(str) + ' $')
    display('Estimated annual Feed-in tariff rebate: '+ Results['FiT_Payment_SolBatt'].astype(int).astype(str) + ' $')
    display('Estimated total annual saving: '+ Results['Annual_Saving_SolBatt'].astype(int).astype(str) + ' $')

    # Estimated daily seasonal load vs PV generation

    LoadSeasonalPattern=pd.read_json(Results['Load_seasonal_pattern_kW'])
    PVSeasonalPattern=pd.read_json(Results['PV_seasonal_pattern_kW'])
    PVBattSeasonalPattern=pd.read_json(Results['PVBatt_seasonal_pattern_kW'])

    LoadSeasonalPattern_Sum=LoadSeasonalPattern.loc[LoadSeasonalPattern[0]==1,[1,2]]
    LoadSeasonalPattern_Fal=LoadSeasonalPattern.loc[LoadSeasonalPattern[0]==2,[1,2]]
    LoadSeasonalPattern_Win=LoadSeasonalPattern.loc[LoadSeasonalPattern[0]==3,[1,2]]
    LoadSeasonalPattern_Spr=LoadSeasonalPattern.loc[LoadSeasonalPattern[0]==4,[1,2]]
    PVSeasonalPattern_Sum=PVSeasonalPattern.loc[PVSeasonalPattern[0]==1,[1,2]]
    PVSeasonalPattern_Fal=PVSeasonalPattern.loc[PVSeasonalPattern[0]==2,[1,2]]
    PVSeasonalPattern_Win=PVSeasonalPattern.loc[PVSeasonalPattern[0]==3,[1,2]]
    PVSeasonalPattern_Spr=PVSeasonalPattern.loc[PVSeasonalPattern[0]==4,[1,2]]
    PVBattSeasonalPattern_Sum=PVBattSeasonalPattern.loc[PVBattSeasonalPattern[0]==1,[1,2]]
    PVBattSeasonalPattern_Fal=PVBattSeasonalPattern.loc[PVBattSeasonalPattern[0]==2,[1,2]]
    PVBattSeasonalPattern_Win=PVBattSeasonalPattern.loc[PVBattSeasonalPattern[0]==3,[1,2]]
    PVBattSeasonalPattern_Spr=PVBattSeasonalPattern.loc[PVBattSeasonalPattern[0]==4,[1,2]]

    trace1 = go.Scatter(
        x=LoadSeasonalPattern_Sum[1],
        y=LoadSeasonalPattern_Sum[2],fill='tozeroy', mode= 'none',name='Load (kW) - Summer')
    trace11 = go.Scatter(
        x=PVSeasonalPattern_Sum[1],
        y=PVSeasonalPattern_Sum[2],fill='tonexty', mode= 'none',name='PV (kW) - Summer')
    trace12 = go.Scatter(
        x=PVBattSeasonalPattern_Sum[1],
        y=PVBattSeasonalPattern_Sum[2], mode= 'lines',name='Net load (with battery) (kW) - Summer')   
    trace13 = go.Scatter(
        x=PVBattSeasonalPattern_Sum[1],
        y=LoadSeasonalPattern_Sum[2]-PVSeasonalPattern_Sum[2], mode= 'lines',name='Net load (kW) - Summer')   
    trace2 = go.Scatter(
        x=LoadSeasonalPattern_Fal[1],
        y=LoadSeasonalPattern_Fal[2],
        xaxis='x2',yaxis='y2',fill='tozeroy', mode= 'none',name='Load (kW) - Autumn')
    trace21 = go.Scatter(
        x=PVSeasonalPattern_Fal[1],
        y=PVSeasonalPattern_Fal[2],
        xaxis='x2',yaxis='y2',fill='tonexty', mode= 'none',name='PV (kW) - Autumn')
    trace22 = go.Scatter(
        x=PVBattSeasonalPattern_Fal[1],
        y=PVBattSeasonalPattern_Fal[2], mode= 'lines',name='Net load (with battery) (kW) - Autumn')  
    trace23 = go.Scatter(
        x=PVBattSeasonalPattern_Fal[1],
        y=LoadSeasonalPattern_Fal[2]-PVSeasonalPattern_Fal[2], mode= 'lines',name='Net load (kW) - Autumn')  
    trace3 = go.Scatter(
        x=LoadSeasonalPattern_Win[1],
        y=LoadSeasonalPattern_Win[2],
        xaxis='x3',yaxis='y3',fill='tozeroy', mode= 'none',name='Load (kW) - Winter')
    trace31 = go.Scatter(
        x=PVSeasonalPattern_Win[1],
        y=PVSeasonalPattern_Win[2],
        xaxis='x3',yaxis='y3',fill='tonexty', mode= 'none',name='PV (kW) - Winter')
    trace32 = go.Scatter(
        x=PVBattSeasonalPattern_Win[1],
        y=PVBattSeasonalPattern_Win[2], mode= 'lines',name='Net load (with battery) (kW) - Winter')  
    trace33 = go.Scatter(
        x=PVBattSeasonalPattern_Win[1],
        y=LoadSeasonalPattern_Win[2]-PVSeasonalPattern_Win[2], mode= 'lines',name='Net load (kW) - Winter')  
    trace4 = go.Scatter(
        x=LoadSeasonalPattern_Spr[1],
        y=LoadSeasonalPattern_Spr[2],
        xaxis='x4',yaxis='y4',fill='tozeroy', mode= 'none',name='Load (kW) - Spring')
    trace41 = go.Scatter(
        x=PVSeasonalPattern_Spr[1],
        y=PVSeasonalPattern_Spr[2],
        xaxis='x4',yaxis='y4',fill='tonexty', mode= 'none',name='PV (kW) - Spring')
    trace42 = go.Scatter(
        x=PVBattSeasonalPattern_Spr[1],
        y=PVBattSeasonalPattern_Spr[2], mode= 'lines',name='Net load (with battery) (kW) - Spring') 
    trace43 = go.Scatter(
        x=PVBattSeasonalPattern_Spr[1],
        y=LoadSeasonalPattern_Spr[2]-PVSeasonalPattern_Spr[2], mode= 'lines',name='Net load (kW) - Spring')  
    
    
    fig = tools.make_subplots(rows=2, cols=2, print_grid=False, subplot_titles=('Summer', 'Autumn',
                                                              'Winter', 'Spring'))
    fig.append_trace(trace1, 1, 1)
    fig.append_trace(trace11, 1, 1)
    fig.append_trace(trace12, 1, 1)
    fig.append_trace(trace13, 1, 1)
    
    fig.append_trace(trace2, 1, 2)
    fig.append_trace(trace21, 1, 2)
    fig.append_trace(trace22, 1, 2)
    fig.append_trace(trace23, 1, 2)

    fig.append_trace(trace3, 2, 1)
    fig.append_trace(trace31, 2, 1)
    fig.append_trace(trace32, 2, 1)
    fig.append_trace(trace33, 2, 1)

    fig.append_trace(trace4, 2, 2)
    fig.append_trace(trace41, 2, 2)
    fig.append_trace(trace42, 2, 2)
    fig.append_trace(trace43, 2, 2)
   
    MaxRange=max(max(LoadSeasonalPattern[2]),max(PVSeasonalPattern[2]))
    MinRange=min(min(LoadSeasonalPattern[2]-PVSeasonalPattern[2]),min(PVBattSeasonalPattern[2]))
    fig['layout']['yaxis1'].update(title='kW',range=[MinRange,MaxRange])
    fig['layout']['yaxis2'].update(title='kW',range=[MinRange,MaxRange])
    fig['layout']['yaxis3'].update(title='kW',range=[MinRange,MaxRange])
    fig['layout']['yaxis4'].update(title='kW',range=[MinRange,MaxRange])

    fig['layout'].update(height=600, width=1000, title='Estimated daily seasonal load vs PV generation',showlegend=True)
    # fig = go.Figure(data, layout)
    iplot(fig)

    
        

'Estimated total annual AC output: 2708 kWh'

'Estimated annual AC output per kW capacity: 902 kWh'

'Estimated annual avoided CO2: 2220 kg CO2'

'PV:'

'Estimated net output to grid (No battery): 1327 kWh'

...

'Battery:'

'Estimated net output to grid: 699 kWh'

'Estimated saving due to not using the grid: 487 $'

'Estimated annual Feed-in tariff rebate: 77 $'

'Estimated total annual saving: 564 $'

In [11]:

fig = { "data": [ { "values": [Results['Annual_PV_Generation']-Results['Est_Annual_PV_export_SolarOnly'],Results['Est_Annual_PV_export_SolarOnly']],
          "labels": ["Self consumption", "Export to the grid"],"domain": {"x": [0, .48]},  "text":["Use", "Export"], "name": "PV Generation (kWh)", 
                       "hoverinfo":"label+percent+value+name",  "hole": .4,  "type": "pie" ,
                   'marker': {'colors': ['rgb(20, 180, 29)',
                                  'rgb(205, 100, 36)']}},  
                     { "values": [Results['New_Bill_SolarOnly'],Results['Saving_due_to_not_using_grid_SolarOnly'], Results['FiT_Payment_SolarOnly']],
          "labels": ["New Bill","Saving due to not using the grid", "Feed in tariff rebate"],
                      "text":["New Bill","Not buying from grid","FiT"],"textposition":"inside","domain": {"x": [.52, 1]},
                      "name": "Annual Bill","hoverinfo":"label+percent+value+name", "hole": .4,"type": "pie",
                   'marker': {'colors': ['rgb(20, 60, 180)',
                                  'rgb(40, 150, 150)',
                                        'rgb(220, 220, 50)']}}],
           "layout": {"title":"Summary of the result (impact of PV)",
                      "annotations": [{ "showarrow": False, "text": 'PV (kWh)',"x": 0.20,"y": 0.5 },
                {  "showarrow": False, "text": "Bill ($)", "x": 0.8, "y": 0.5}] }}
iplot(fig)

    
fig = { "data": [ { "values": [Results['Annual_PV_Generation']-Results['Est_Annual_PV_export_SolBatt'],Results['Est_Annual_PV_export_SolBatt']],
          "labels": ["Self consumption", "Export to the grid"],"domain": {"x": [0, .48]},  "text":["Use", "Export"], "name": "PV Generation (kWh)", 
                       "hoverinfo":"label+percent+value+name",  "hole": .4,  "type": "pie",
                   'marker': {'colors': ['rgb(20, 180, 29)',
                                  'rgb(205, 100, 36)']}},  
                     { "values": [Results['New_Bill_SolBatt'], Results['Saving_due_to_not_using_grid_SolBatt'], Results['FiT_Payment_SolBatt']],
          "labels": ["New Bill","Saving due to not using the grid", "Feed in tariff rebate"],
                      "text":["New Bill","Not buying from grid","FiT"],"textposition":"inside","domain": {"x": [.52, 1]},
                      "name": "Annual Bill","hoverinfo":"label+percent+value+name", "hole": .4,"type": "pie",
                   'marker': {'colors': ['rgb(20, 60, 180)',
                                  'rgb(40, 150, 150)',
                                        'rgb(220, 220, 50)']}}],
           "layout": {"title":"Summary of the result (impact of PV and Battery)",
                      "annotations": [{ "showarrow": False, "text": 'PV (kWh)',"x": 0.20,"y": 0.5 },
                {  "showarrow": False, "text": "Bill ($)", "x": 0.8, "y": 0.5}] }}
iplot(fig)